In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, glob, os
import scipy.stats as stats, scipy.io as sio


### create spikes df

In [ ]:
def get_ID_spikes_dict(clustIDs, spikes):
    ''' return dict with keys=unique clusters, and vals = list of corresponding spikes '''
    
    ID_spikes_dict = {}
    for clustID, spike in zip(clustIDs, spikes):

        if clustID in [0, 99999999]: continue

        if clustID not in ID_spikes_dict: ID_spikes_dict[clustID] = [] # init

        ID_spikes_dict[clustID].append(spike)

    return ID_spikes_dict

In [115]:
samp_rate = 1000000
spikes_df = []

data_dir = '../../results/202512/osort_mat/sort/5'
for channel_num in range(len(os.listdir(data_dir))):

    channel_mat = sio.loadmat(f'{data_dir}/A{channel_num+1}_sorted_new.mat')

    # get (1 * n_spikes) clusterID & spiketimes
    clust_IDs = channel_mat['assignedNegative'][0]
    spikes = channel_mat['newTimestampsNegative'][0]

    # get clusterID: [spikes]
    ID_spikes_dict = get_ID_spikes_dict(clust_IDs, spikes)

    # 1 row per clustID
    clust_df = pd.DataFrame([
        {
            "channel": channel_num + 1,
            "clusterID": clustID,
            "spikes": spikes,
            "#spikes": len(spikes),
            "avgFR": len(spikes) / ((spikes[-1] - spikes[0]) / samp_rate),
        }
        for clustID, spikes in ID_spikes_dict.items()
    ])
    spikes_df.append(clust_df)

    # if channel_num > 10: break

spikes_df = pd.concat(spikes_df, ignore_index=True)
spikes_df


,channel,clusterID,spikes,#spikes,avgFR
0,1,612,"[18085633.333333336, 23637800.0, 25171666.6666...",1062,0.670538
1,1,596,"[33789566.66666667, 34336866.66666667, 3711156...",318,0.203225
2,2,1583,"[4070566.666666667, 12215633.333333334, 123644...",5454,3.372888
3,2,1502,"[61635733.333333336, 110395233.33333334, 13552...",236,0.152713
4,2,1543,"[287371366.6666667, 327906366.6666667, 4803090...",247,0.185968
...,...,...,...,...,...
85,30,156,"[2322066.666666667, 2338566.666666667, 2393566...",402,0.247405
86,31,182,"[2309366.666666667, 2322066.666666667, 2343133...",506,0.311551
87,31,188,"[2325900.0, 3751133.3333333335, 12928766.66666...",479,0.295834
88,32,167,"[2275766.666666667, 3381300.0000000005, 796643...",480,0.296040
